In [1]:
import pandas as pd
import backtesting as bt
import pandas_ta as ta
import numpy as np
import datetime as dt
from IPython.display import clear_output
import breeze_client as bc
import yfinance as yf
import time


In [8]:
df1 = pd.read_csv('data/NIFTY/NIFTY_options_23000_13-06-2024_1minute_10d.csv')
df1

,Close,datetime,exchange_code,expiry_date,High,Low,Open,open_interest,product_type,right,stock_code,strike_price,Volume
0,372.90,2024-05-30 09:15:00,NFO,13-JUN-2024,410.00,364.10,410.00,417975,Options,Call,NIFTY,23000.0,4975
1,368.00,2024-05-30 09:16:00,NFO,13-JUN-2024,372.00,366.00,370.00,417975,Options,Call,NIFTY,23000.0,1725
2,372.00,2024-05-30 09:17:00,NFO,13-JUN-2024,372.90,366.15,368.00,417975,Options,Call,NIFTY,23000.0,1725
3,378.70,2024-05-30 09:18:00,NFO,13-JUN-2024,378.70,370.00,372.00,420550,Options,Call,NIFTY,23000.0,2600
4,382.80,2024-05-30 09:19:00,NFO,13-JUN-2024,390.20,380.00,380.00,420550,Options,Call,NIFTY,23000.0,3650
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620,402.35,2024-06-07 15:25:00,NFO,13-JUN-2024,405.15,402.30,403.30,3919850,Options,Call,NIFTY,23000.0,76225
2621,399.05,2024-06-07 15:26:00,NFO,13-JUN-2024,402.20,399.05,402.10,3919850,Options,Call,NIFTY,23000.0,93100
2622,404.25,2024-06-07 15:27:00,NFO,13-JUN-2024,404.60,397.50,399.05,3919850,Options,Call,NIFTY,23000.0,81025
2623,407.80,2024-06-07 15:28:00,NFO,13-JUN-2024,408.00,403.05,404.50,3775350,Options,Call,NIFTY,23000.0,60275


In [9]:
df_nifty = pd.read_csv('data/NIFTY/NIFTY_1minute_10d.csv')
df2 = df_nifty[['Close','datetime','High','Low','Open','Volume']]
df2

,Close,datetime,High,Low,Open,Volume
0,22617.45,2024-05-30 09:09:00,22617.45,22617.45,22617.45,0
1,22617.45,2024-05-30 09:10:00,22617.45,22617.45,22617.45,0
2,22617.45,2024-05-30 09:11:00,22617.45,22617.45,22617.45,0
3,22617.45,2024-05-30 09:12:00,22617.45,22617.45,22617.45,0
4,22617.45,2024-05-30 09:13:00,22617.45,22617.45,22617.45,0
...,...,...,...,...,...,...
2662,23282.75,2024-06-07 15:25:00,23288.90,23281.70,23287.40,0
2663,23279.40,2024-06-07 15:26:00,23282.80,23274.40,23282.20,0
2664,23284.90,2024-06-07 15:27:00,23284.90,23275.05,23279.40,0
2665,23271.55,2024-06-07 15:28:00,23283.55,23271.00,23282.80,0


In [ ]:
# Merge the DataFrames on column 'A' with an indicator
merged_df = pd.merge(df1, df2, on='datetime', how='outer', indicator=True)

# Filter rows where column 'A' is not common in both DataFrames
non_common_rows = merged_df[merged_df['_merge'] != 'both']

# Drop the indicator column and unnecessary columns from the merge
non_common_rows = non_common_rows.drop(columns=['_merge'])

non_common_rows

In [11]:
df = pd.merge(df1,df2,how='inner',on='datetime',suffixes=('','_nifty'))
df


,Close,datetime,exchange_code,expiry_date,High,Low,Open,open_interest,product_type,right,stock_code,strike_price,Volume,Close_nifty,High_nifty,Low_nifty,Open_nifty,Volume_nifty
0,372.90,2024-05-30 09:15:00,NFO,13-JUN-2024,410.00,364.10,410.00,417975,Options,Call,NIFTY,23000.0,4975,22653.90,22674.20,22635.20,22635.20,0
1,368.00,2024-05-30 09:16:00,NFO,13-JUN-2024,372.00,366.00,370.00,417975,Options,Call,NIFTY,23000.0,1725,22640.05,22651.80,22637.45,22651.30,0
2,372.00,2024-05-30 09:17:00,NFO,13-JUN-2024,372.90,366.15,368.00,417975,Options,Call,NIFTY,23000.0,1725,22647.45,22652.40,22635.50,22635.95,0
3,378.70,2024-05-30 09:18:00,NFO,13-JUN-2024,378.70,370.00,372.00,420550,Options,Call,NIFTY,23000.0,2600,22670.30,22670.30,22641.95,22645.15,0
4,382.80,2024-05-30 09:19:00,NFO,13-JUN-2024,390.20,380.00,380.00,420550,Options,Call,NIFTY,23000.0,3650,22684.00,22688.55,22667.75,22671.25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620,402.35,2024-06-07 15:25:00,NFO,13-JUN-2024,405.15,402.30,403.30,3919850,Options,Call,NIFTY,23000.0,76225,23282.75,23288.90,23281.70,23287.40,0
2621,399.05,2024-06-07 15:26:00,NFO,13-JUN-2024,402.20,399.05,402.10,3919850,Options,Call,NIFTY,23000.0,93100,23279.40,23282.80,23274.40,23282.20,0
2622,404.25,2024-06-07 15:27:00,NFO,13-JUN-2024,404.60,397.50,399.05,3919850,Options,Call,NIFTY,23000.0,81025,23284.90,23284.90,23275.05,23279.40,0
2623,407.80,2024-06-07 15:28:00,NFO,13-JUN-2024,408.00,403.05,404.50,3775350,Options,Call,NIFTY,23000.0,60275,23271.55,23283.55,23271.00,23282.80,0


In [13]:
def supertrend(data,length=10,multiplier=3):

        super = ta.supertrend(data.High_nifty.s,data.Low_nifty.s,data.Close_nifty.s,length,multiplier) 

        return super.to_numpy().T[0]



def atr(dfpl,length):

        atr = ta.atr(dfpl.High.s, dfpl.Low.s, dfpl.Close.s, length)

        return atr.to_numpy()





In [52]:



class SuperTrend(bt.Strategy):

    multiplier = 3
    period = 9
    atr_length = 7

    
    def init(self):
        self.super = self.I(supertrend,self.data,self.period,self.multiplier) 
        self.ATR = self.I(atr,self.data,self.atr_length)

    
    def next(self):
        slatr = 1.2*self.ATR[-1]
        TPSLRatio = 1.5

        current_price = self.data.Close[-1]


        n_current_price = self.data.Close_nifty[-1]
        last_price = self.data.Close_nifty[-2]
        current_trend = self.super[-1]
        last_trend = self.super[-2]
        
        #buy signal
        if (last_trend > last_price) and (current_trend < n_current_price):
            self.position.close()
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy()
            
            
            
                

        if (last_trend < last_price) and (current_trend > n_current_price):
            self.position.close()
            
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell()

                
        

        


            


        
    

In [53]:
eng = bt.Backtest(resampled_df,SuperTrend,cash=25000)

In [54]:
stats = eng.run()
stats

Start                     2024-06-07 09:15:00
End                       2024-06-07 15:29:30
Duration                      0 days 06:14:30
Exposure Time [%]                   21.066667
Equity Final [$]                 22124.211783
Equity Peak [$]                       26542.9
Return [%]                         -11.503153
Buy & Hold Return [%]              311.793612
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -26.932189
Avg. Drawdown [%]                  -15.050695
Max. Drawdown Duration        0 days 05:36:00
Avg. Drawdown Duration        0 days 02:49:30
# Trades                                   22
Win Rate [%]                        40.909091
Best Trade [%]                       7.594311
Worst Trade [%]                     -6.345709
Avg. Trade [%]                    

In [55]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2448', ...)

In [18]:
def optim_func(series):

    if series['Win Rate [%]'] > 65 and series['# Trades'] > 30:
        return series['Return [%]']
    
    


    else:
        return -1000000

opt_stats,heatmap = eng.optimize(
    multiplier = range(1,10),
    period = range(3,50),
    
    maximize = 'Win Rate [%]',
    
    return_heatmap = True,
    #max_tries = 1000

)
opt_stats

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 423 configurations.
  output = _optimize_grid()


Start                     2024-06-07 09:15:00
End                       2024-06-07 15:29:30
Duration                      0 days 06:14:30
Exposure Time [%]                        89.6
Equity Final [$]                      25639.2
Equity Peak [$]                       42085.3
Return [%]                             2.5568
Buy & Hold Return [%]              311.793612
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -49.067727
Avg. Drawdown [%]                  -17.530593
Max. Drawdown Duration        0 days 03:37:00
Avg. Drawdown Duration        0 days 00:47:30
# Trades                                    4
Win Rate [%]                             50.0
Best Trade [%]                      75.035063
Worst Trade [%]                     -26.19469
Avg. Trade [%]                    

In [20]:
eng.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1713', ...)

In [19]:
opt_stats.plot()

TypeError: no numeric data to plot

In [29]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,20,22,22347.800781,22408.500000,-60.699219,-0.002716,2024-04-25 10:55:00+05:30,2024-04-25 11:05:00+05:30,0 days 00:10:00
1,1,46,53,22460.849609,22479.750000,18.900391,0.000841,2024-04-25 13:05:00+05:30,2024-04-25 13:40:00+05:30,0 days 00:35:00
2,-1,91,114,22518.099609,22499.050781,19.048828,0.000846,2024-04-26 10:35:00+05:30,2024-04-26 12:30:00+05:30,0 days 01:55:00
3,1,151,152,22490.849609,22511.050781,20.201172,0.000898,2024-04-29 09:20:00+05:30,2024-04-29 09:25:00+05:30,0 days 00:05:00
4,-1,292,292,22720.800781,22702.650391,18.150391,0.000799,2024-04-30 14:50:00+05:30,2024-04-30 14:50:00+05:30,0 days 00:00:00
5,1,306,311,22655.800781,22677.400391,21.599609,0.000953,2024-05-02 09:45:00+05:30,2024-05-02 10:10:00+05:30,0 days 00:25:00
6,-1,361,375,22647.500000,22769.099609,-121.599609,-0.005369,2024-05-02 14:20:00+05:30,2024-05-03 09:15:00+05:30,0 days 18:55:00
7,1,376,377,22769.599609,22789.750000,20.150391,0.000885,2024-05-03 09:20:00+05:30,2024-05-03 09:25:00+05:30,0 days 00:05:00
8,-1,387,388,22682.599609,22661.400391,21.199219,0.000935,2024-05-03 10:15:00+05:30,2024-05-03 10:20:00+05:30,0 days 00:05:00
9,1,445,447,22474.900391,22495.599609,20.699219,0.000921,2024-05-03 15:05:00+05:30,2024-05-03 15:15:00+05:30,0 days 00:10:00
